In [1]:
%pip install requests azure-identity msgraph-sdk neo4j graphdatascience

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 11.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 13.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 8.5.0
    Uninstalling importlib_metadata-8.5.0:
      Successfully uninstalled importlib_metadata-8.5.0
Note: you may need to restart the kernel to use updated packages.


In [7]:
import asyncio
import configparser
from msgraph.generated.models.o_data_errors.o_data_error import ODataError

from configparser import SectionProxy
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.user_item_request_builder import UserItemRequestBuilder
from msgraph.generated.sites.sites_request_builder import SitesRequestBuilder

In [29]:
class Graph:
    settings: SectionProxy
    device_code_credential: DeviceCodeCredential
    user_client: GraphServiceClient

    def __init__(self,config: SectionProxy):
        self.settings = config
        client_id = self.settings['clientId']
        tenant_id = self.settings['tenantId']
        graph_scopes = self.settings['graphUserScopes'].split(' ')

        self.device_code_credential = DeviceCodeCredential(client_id, tenant_id=tenant_id)
        self.user_client = GraphServiceClient(self.device_code_credential, graph_scopes)

    async def get_user_token(self):
        graph_scopes = self.settings['GraphUserScopes']
        access_token = self.device_code_credential.get_token(graph_scopes)
        return access_token.token
    
    async def get_user(self):
        # only request specific properties using $select
        query_params = UserItemRequestBuilder.UserItemRequestBuilderGetQueryParameters(
            select=['displayName', 'mail', 'userPrincipalName']
        )
        request_config = UserItemRequestBuilder.UserItemRequestBuilderGetRequestConfiguration(
            query_parameters=query_params
        )
        user = await self.user_client.me.get(request_configuration=request_config)
        return user
    
    async def get_default_site(self):
        query_params = UserItemRequestBuilder.UserItemRequestBuilderGetQueryParameters(
            select=["siteCollection","webUrl"]
        )
        request_config = UserItemRequestBuilder.UserItemRequestBuilderGetRequestConfiguration(
            query_parameters=query_params
        )
        site = await self.user_client.sites.get(request_configuration=request_config)
        return site

In [30]:
async def greet_user(graph: Graph): 
    user = await graph.get_user()
    if user:
        print(f'Hello {user.display_name}')
        print(f'Email: {user.mail or user.user_principal_name}')

In [31]:
config = configparser.ConfigParser()
config.read(['config.cfg'])
azure_settings = config['azure']

graph: Graph = Graph(azure_settings)

In [18]:
await greet_user(graph)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code SDCWTAEED to authenticate.
Hello Muhammad Arif Wicaksana
Email: arif@muarwi.onmicrosoft.com


In [32]:
async def get_site(graph: Graph):
    site = await graph.get_default_site()
    if site:
        print(site.display_name)

In [28]:
await get_site(graph)

AttributeError: 'SiteCollectionResponse' object has no attribute 'display_name'